# ALS

In [ ]:
##ALS functions
function findD(n,m,R;maxdnumber=20)#d  at most is 20
    costs=zeros(maxdnumber)
    for i in 1:maxdnumber 
        U,V=ALS(n,m,R,i)
        costs[i]=ALSCOST(U,V,R,0.1)
    end
    p=plot(1:maxdnumber,costs,title = "findD")
    savefig(p,"C:\\Users\\hossein rahmani\\Desktop\\factorization.png")
    display(p)
end
# U d*n and V is d*m matrix
function ALS(n,m,R,d;maxitr=40,landa=0.1,tre=1e-5)
  (U,V)=Initialize(n,d,m,R)
  Costprev=Inf
    for iteration in 1:maxitr
        
        for u in 1:n 
            U[:,u]=computeU(d,u,R,V,landa)
        end
        ##
        for i in 1:m 
          V[:,i]= computeV(d,i,R,U,landa)
        end
        C = ALSCOST(U,V,R,landa)
        println("Iteration ", iteration, ": costoptimization = ", C, ".")
  if iteration > 1 && abs(C - Costprev) < tre * C
    return U, V,C
  end
    Costprev = C
    end
    return U,V
end
function ALSCOST(U,V,R,landa)
    cost=0
    n=size(R,1)
    m=size(R,2)
    for u in 1:n
        for i in 1:m 
            if (R[u,i]!=0)
                cost=cost+(R[u,i]-U[:,u]'*V[:,i])^2
            end
        end
    end
    for u in 1:n 
        cost=cost+landa*normsqu(U[u])
    end
    for i in 1:m 
        cost=cost+landa*normsqu(V[i])
    end
    return cost
end
function computeV(d,i,R,U,landa)
    before=zeros(d,d)
    b=zeros(d)
    for j in 1:n
        if (R[j,i]!=0)
            before=before+U[:,j]*U[:,j]'+landa*Matrix{Float64}(I, d, d)
            b=b+R[j,i]*U[:,j]
        end 
    end
    return inv(before)*b
end
function computeU(d,u,R,V,landa)
    before=zeros(d,d)
    b=zeros(d)
    for i in 1:m 
        if (R[u,i]!=0)
            before=before+V[:,i]*V[:,i]'+landa*Matrix{Float64}(I, d, d)
            b=b+R[u,i]*V[:,i]
        end
    end
   return inv(before)*b
end
function Initialize(n,d,m,R)
    U=zeros(d,n)
    V=zeros(d,m)
    for i in 1:d 
        for j in 1:n 
           U[i,j]= rand(Uniform(0,2))
        end
        for k in 1:m 
            V[i,k]=rand(Uniform(0,2))
        end
    end
    return U,V
end

function normsqu(u)
    res=0
    for s in 1:length(u) 
        res=res+u[s]^2
    end
    return res
end

In [ ]:
R = readdlm("C:\\Users\\hossein rahmani\\Desktop\\HW8\\data\\Q4\\agent-food.txt",' ')
n=size(R,1)
m=size(R,2)
findD(n,m,R)
#Az nemodar mifahmim d in hast hala U,V ro migirim

In [ ]:
#Az nemodar mifahmim d in hast hala U,V ro migirim
d=3
U,V=ALS(n,m,R,d)

# similar people

In [ ]:
#b midterm

similars(U,n,57,10)

function similarity(u1,u2)#similar to u1
   return dot(u1,u2)/norm(u2)
end
function similars(U,n,i,k)#find k most similar people to person ith.
    dict = Dict(j =>similarity(U[:,i],U[:,j])  for j in 1:n)
    b= sort(collect(dict), by = x->x[2])
    for j in 2:k+1
        println(string(b[j][2]," ",b[j][1]))
    end 
    
end

In [ ]:
 #aval meghdar similarity bad shomare fard nazdic be nafar 57
2.4434680508266755 327
2.4444751269334497 485
2.477347311740324 1805
2.574742676526837 65
2.8505637961019774 1452
2.933821450485836 1190
2.99616908928197 1612
3.219503430108774 1474
3.241367987351699 106
3.2460771434584763 1429

# similar foods

In [ ]:
elbow(V)
# از شکل درمیابیم که باید به دودسته تقسیم شود.
t=2
(assignments, reps)=kmeans(V,2)

In [ ]:
assigments=[
3-element Vector{Int64}:
 1
 1
 2
    ]

In [ ]:
function avg(v)
    sum(v)/length(v)
end
function Jclust(x,reps,assignments)
    avg(norm(x[:,i]- reps[assignments[i]] )^2 for i=1:size(x,2))
end
function vectorsum(x,n,group)
    res=zeros(n)
    for i in 1:length(group) 
        res=res+x[:,group[i]]
    end
    return res
end
function kmeans(x,k;maxitr=30,tre=1e-5)
    N=size(x,2)
    n=size(x,1)
    distances=zeros(N)
    reps=[zeros(n) for j=1:k]
    assignments=[rand(1:k) for i=1:N]  #in which group
    Jprevious= Inf
    for iteration in 1:maxitr
       for j = 1:k
            group = [i for i=1:N if assignments[i] == j]
            reps[j] = vectorsum(x,n,group) / length(group)
            end
            # For each x[i], find distance to the nearest representative

            for i = 1:N
               (distances[i], assignments[i]) =
               findmin([norm(x[:,i] - reps[j]) for j = 1:k])
               end
      
               J = norm(distances)^2 / N
               println("Iteration ", iteration, ": Jclust = ", J, ".")
         if iteration > 1 && abs(J - Jprevious) < tre * J
           return assignments, reps
         end
        Jprevious = J
    end 
    return assignments, reps
end
function elbow(C;maxk=3)
    cost=zeros(maxk)

    for i in 1:maxk
       (assignments,reps) =kmeans(C,i)
       cost[i]=Jclust(C,reps,assignments)
    end
    p=plot(1:maxk,cost,title = "elbow_Pmidterm")
    savefig(p,"C:\\Users\\hossein rahmani\\Desktop\\elbowforfood.png")
    display(p)
end
